In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import os

# 設定裝置 (如果有 GPU 則使用 cuda)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 1. 定義資料預處理 (調整大小 + 轉 Tensor + 正規化)
transform = transforms.Compose([
    transforms.Resize((224, 224)), # 統一圖像大小
    transforms.ToTensor(),         # 轉為 PyTorch Tensor (0-1)
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # ImageNet 標準
])

# 2. 指定 Kaggle 資料集路徑
# 請確認路徑是否正確，通常 Kaggle input 結構如下
dataset_path = '/home/whsung8451/.cache/kagglehub/datasets/briscdataset/brisc2025/versions/6/brisc2025/classification_task/train'

# 載入完整資料集
full_dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# 3. 切分 訓練集 (80%) 與 驗證集 (20%)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# 建立 DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

print(f"訓練集數量: {len(train_dataset)}, 驗證集數量: {len(val_dataset)}")
print(f"類別: {full_dataset.classes}") # 應顯示 ['glioma', 'meningioma', 'no_tumor', 'pituitary']

Using device: cuda
訓練集數量: 4000, 驗證集數量: 1000
類別: ['glioma', 'meningioma', 'no_tumor', 'pituitary']


In [2]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=4):
        super(SimpleCNN, self).__init__()
        
        # 特徵提取層 (Feature Extractor)
        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), # 224 -> 112
            
            # Block 2
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # 112 -> 56
            
            # Block 3
            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)  # 56 -> 28
        )
        
        # 分類層 (Classifier)
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 28 * 28, 512), #計算方式: Channel數 * 寬 * 高
            nn.ReLU(),
            nn.Dropout(0.5), # 防止過擬合
            nn.Linear(512, num_classes) # 輸出 4 個類別
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# 初始化模型
model = SimpleCNN(num_classes=4).to(device)
print(model)

SimpleCNN(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=100352, out_features=512, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=512, out_features=4, bias=True)
  )
)


In [3]:
# 設定 Loss 和 Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 訓練函式
def train_model(model, train_loader, val_loader, epochs=10):
    train_losses, val_accuracies = [], []
    
    for epoch in range(epochs):
        model.train() # 設定為訓練模式
        running_loss = 0.0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()           # 梯度歸零
            outputs = model(images)         # 前向傳播
            loss = criterion(outputs, labels) # 計算 Loss
            loss.backward()                 # 反向傳播
            optimizer.step()                # 更新權重
            
            running_loss += loss.item()
        
        # 每個 Epoch 結束後進行驗證
        model.eval() # 設定為評估模式
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        accuracy = 100 * correct / total
        avg_loss = running_loss / len(train_loader)
        
        train_losses.append(avg_loss)
        val_accuracies.append(accuracy)
        
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}, Val Accuracy: {accuracy:.2f}%")
    
    return train_losses, val_accuracies

# 開始訓練 (建議先跑 10-15 epochs 看收斂情況)
history = train_model(model, train_loader, val_loader, epochs=15)

Epoch [1/15], Loss: 0.9663, Val Accuracy: 81.70%
Epoch [2/15], Loss: 0.3888, Val Accuracy: 88.30%
Epoch [3/15], Loss: 0.2285, Val Accuracy: 90.20%
Epoch [4/15], Loss: 0.1461, Val Accuracy: 88.40%
Epoch [5/15], Loss: 0.1021, Val Accuracy: 91.70%
Epoch [6/15], Loss: 0.0783, Val Accuracy: 92.40%
Epoch [7/15], Loss: 0.0428, Val Accuracy: 91.90%
Epoch [8/15], Loss: 0.0336, Val Accuracy: 92.10%
Epoch [9/15], Loss: 0.0304, Val Accuracy: 91.70%
Epoch [10/15], Loss: 0.0337, Val Accuracy: 93.30%
Epoch [11/15], Loss: 0.0176, Val Accuracy: 93.00%
Epoch [12/15], Loss: 0.0214, Val Accuracy: 92.90%
Epoch [13/15], Loss: 0.0449, Val Accuracy: 92.00%
Epoch [14/15], Loss: 0.0256, Val Accuracy: 92.40%
Epoch [15/15], Loss: 0.0204, Val Accuracy: 92.50%
